In [1]:
# py: /src/rag.py
from typing import List, Dict, Any
from config import MODELSCOPE_API_KEY, ST_DIR, CE_DIR, VECTOR_DB_DIR
from openai import OpenAI
from sentence_transformers import SentenceTransformer, CrossEncoder
import chromadb
import warnings

warnings.filterwarnings('ignore', message='.*initialized.*')


class LawRAGSystem:
    def __init__(self):
        print("加载 embedding 模型...")
        self.embedding_model = SentenceTransformer(str(ST_DIR))

        print("加载向量数据库...")
        self.client = chromadb.PersistentClient(path=VECTOR_DB_DIR)
        self.collection = self.client.get_collection("laws_collection")

        print("初始化大模型客户端...")
        self.llm = OpenAI(
            api_key=MODELSCOPE_API_KEY,
            base_url="https://api-inference.modelscope.cn/v1/"
        )

        self._reranker = None
        self.history = []

    @property
    def reranker(self):
        if self._reranker is None:
            print("加载重排序模型 bge-reranker-base ...")
            self._reranker = CrossEncoder(str(CE_DIR))
        return self._reranker

    def embed(self, text: str):
        return self.embedding_model.encode(text, normalize_embeddings=True).tolist()

    def retrieve(self, query: str, k=10):
        emb = self.embed(query)
        result = self.collection.query(query_embeddings=[emb], n_results=k)
        return result["documents"][0]

    def rerank(self, query: str, docs: List[str], k=5):
        pairs = [(query, d) for d in docs]
        scores = self.reranker.predict(pairs)
        ranked = sorted(zip(docs, scores), key=lambda x: x[1], reverse=True)
        return [d for d, _ in ranked[:k]]

    def answer(self, query: str):
        retrieved = self.retrieve(query)
        reranked = self.rerank(query, retrieved)

        context = "\n\n".join(reranked)

        messages = [
            {"role": "system",
             "content": f"请使用以下法律条文回答问题，不得编造：\n\n{context}"},
            {"role": "user", "content": query}
        ]

        resp = self.llm.chat.completions.create(
            model="Qwen/Qwen2.5-Coder-32B-Instruct",
            messages=messages
        )

        answer = resp.choices[0].message.content
        self.history.append((query, answer))

        return answer, reranked

    def show_history(self):
        if not self.history:
            print("暂无对话历史。")
            return
        print("\n=== 对话历史 ===")
        for i, (q, a) in enumerate(self.history, 1):
            print(f"[{i}] 用户：{q}")
            print(f"     回答：{a}\n")

    def clear(self):
        self.history.clear()
        print("对话历史已清空。")


def interactive_dialogue():
    print("🎉 法律 RAG 系统启动！")
    print("输入：quit / exit 退出")
    print("输入：clear 清空历史")
    print("输入：history 查看历史\n")

    rag = LawRAGSystem()

    while True:
        try:
            q = input("\n👤 你：").strip()
            if q.lower() in ["quit", "exit"]:
                print("再见！")
                break
            if q.lower() == "clear":
                rag.clear()
                continue
            if q.lower() == "history":
                rag.show_history()
                continue
            if not q:
                continue

            print("\n🤖 回答中...\n")
            answer, ref = rag.answer(q)

            print("📝 回答：")
            print(answer)

            print("\n📚 相关条文：")
            for i, c in enumerate(ref, 1):
                display = c[:150] + "..." if len(c) > 150 else c
                print(f"  {i}. {display}")

        except KeyboardInterrupt:
            print("\n退出系统。")
            break
        except Exception as e:
            print(f"❌ 错误：{e}")


if __name__ == "__main__":
    interactive_dialogue()


🎉 法律 RAG 系统启动！
输入：quit / exit 退出
输入：clear 清空历史
输入：history 查看历史

加载 embedding 模型...
加载向量数据库...
初始化大模型客户端...

🤖 回答中...

加载重排序模型 bge-reranker-base ...


The tokenizer you are loading from 'C:\Users\lxx\Documents\Codes\PYTHONs\laws-rag\models\cross-encoder\bge-reranker-base' with an incorrect regex pattern: https://huggingface.co/mistralai/Mistral-Small-3.1-24B-Instruct-2503/discussions/84#69121093e8b480e709447d5e. This will lead to incorrect tokenization. You should set the `fix_mistral_regex=True` flag when loading this tokenizer to fix this issue.


📝 回答：
我是根据您提供的法律条文设计的助手，用于帮助解答与这些条文相关的问题。如果您有任何具体问题，请随时提问。

📚 相关条文：
  1. 第十五章 附 则 | 第二百六十五条 本法下列用语的含义： （一）高级管理人员，是指公司的经理、副经理、财务负责人，上市公司董事会秘书和公司章程规定的其他人员。 （二）控股股东，是指其出资额占有限责任公司资本总额超过百分之五十或者其持有的股份占股份有限公司股本总额超过百分之五十的股东；出资额或者持有...
  2. 第三章 有限责任公司的设立和组织机构, 第一节 设 立 | 第五十六条 有限责任公司应当置备股东名册，记载下列事项： （一）股东的姓名或者名称及住所； （二）股东认缴和实缴的出资额、出资方式和出资日期； （三）出资证明书编号； （四）取得和丧失股东资格的日期。 记载于股东名册的股东，可以依股东名册主...
  3. 第三章 有限责任公司的设立和组织机构, 第二节 组织机构 | 第五十八条 有限责任公司股东会由全体股东组成。股东会是公司的权力机构，依照本法行使职权。
  4. 第五章 股份有限公司的设立和组织机构, 第二节 股东会 | 第一百一十八条 股东委托代理人出席股东会会议的，应当明确代理人代理的事项、权限和期限；代理人应当向公司提交股东授权委托书，并在授权范围内行使表决权。
  5. 第五章 股份有限公司的设立和组织机构, 第二节 股东会 | 第一百一十一条 股份有限公司股东会由全体股东组成。股东会是公司的权力机构，依照本法行使职权。

退出系统。
